Clean Code

In [11]:
# library needed
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re
from sklearn import preprocessing
import scipy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# Parsing

In [12]:
def parsing(df):
    nacolumns = df.isna().astype('int').iloc[:,:-1]
    columns = ["company_profile","description","requirements","benefits"]
    one_hot_encoding_columns = ['employment_type', "required_education", "required_experience","function.", "industry"]
    length = []
    for column in columns:
        df[(str(column) + "_length")] = df[column].apply(lambda x: len(x) if x == x else 0)
    df = pd.get_dummies(df, columns = one_hot_encoding_columns)
    return df.iloc[:, 9:]
    return pd.concat([df.iloc[:, 9:],nacolumns],axis=1)

In [13]:
jobs = pd.read_csv("job_training_data.csv")
jobs = parsing(jobs)
feature_matrix = jobs

# PCA

In [14]:
data_normalized = preprocessing.scale(feature_matrix,with_mean=True,with_std=True)

In [15]:
pca = PCA()
pca.fit(data_normalized)
components = pca.components_[0:np.where(pca.explained_variance_ratio_.cumsum() > 0.9)[0][0]+1,:]
pca_features = np.matmul(data_normalized,np.transpose(components))

In [16]:
feature_matrix = pca_features

X = pca_features
y = jobs.fraudulent
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)
rfm = RandomForestClassifier(n_estimators = 200,n_jobs=6)

rfm = RandomForestClassifier(n_estimators = 200,n_jobs=6)
rfm.fit(X_train,y_train)
rfm.score(X_train,y_train)
y_pred_rfm=rfm.predict(X_test)

In [17]:
sklearn.metrics.confusion_matrix(y_test,y_pred_rfm)

array([[1543,    0],
       [  23,   43]], dtype=int64)